not simple linear regression
    2 or more models
    


In [173]:
import pandas as pd
import statsmodels.formula.api as stats
import statsmodels.api as sm
import numpy as np
import math
import sklearn.metrics as sk
from bs4 import BeautifulSoup
import requests
import json

In [8]:
def GetPageData(URL):
    page = requests.get(URL)
    return(page)

In [360]:
URL = 'https://barttorvik.com/trank.php?year=2010&sort=&top=0&conlimit=All#'

TeamData = {}

page = GetPageData(URL)
soup = BeautifulSoup(page.content, 'html.parser')

#College Name
Names = []
TeamName = soup.find_all(class_='teamname')
for teams in TeamName:
    Names.append(teams.text.split('(')[0])
TeamData['Name']=Names

#Team Record
Games=[]
GamesPlayed = soup.find_all(class_='6 mobileout')
for games in GamesPlayed:
    Games.append(games.text)    
Games.pop(0)
TeamData['GamesPlayed']=Games


ConferenceRecord= []
TotalRecord = []
Record = soup.find_all(class_='5')
for rec in Record:
    if(rec.text != 'Rec'):
        TotalRecord.append("'"+rec.find('a').text)
        ConferenceRecord.append("'"+rec.find(class_='lowrow').text)
TeamData['ConfRecord']=ConferenceRecord
TeamData['TotalRecord']=TotalRecord

#Offensive Efficency
OffEff = []
OffensiveEfficencey = soup.find_all(class_='1')
for eff in OffensiveEfficencey:
    if(eff.text != 'AdjOE'):
        OffEff.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
TeamData['OffEff']=OffEff

#Deffensive Efficency
DefEff = []
DeffensiveEfficencey = soup.find_all(class_='2')
for eff in DeffensiveEfficencey:
    if(eff.text != 'AdjDE'):
        DefEff.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
TeamData['DefEff']=DefEff

#Turnovers
Turnovers = []
Turnover = soup.find_all(class_='11')
for eff in Turnover:
    try:
        Turnovers.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['Turnovers']=Turnovers

#ForcedTurnovers
fTurnovers = []
fTurnover = soup.find_all(class_='12')
for eff in fTurnover:
    try:
        fTurnovers.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['ForcedTurnovers']=fTurnovers


#2%FGOffensive
Perc2Of = []
Perc2Ofo = soup.find_all(class_='16')
for eff in Perc2Ofo:
    try:
        Perc2Of.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['2PFGOff']=Perc2Of


#2%FGDeffensive
Perc2D = []
Perc2Ofd = soup.find_all(class_='17')
for eff in Perc2Ofd:
    try:
        Perc2D.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['2PFGDef']=Perc2D


#3%FGOffensive
Perc3Of = []
Perc3Ofo = soup.find_all(class_='18')
for eff in Perc3Ofo:
    try:
        Perc3Of.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['3PFGOff']=Perc3Of

#3%FGDeffensive
Perc3D = []
Perc3Ofd = soup.find_all(class_='19')
for eff in Perc3Ofd:
    try:
        Perc3D.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['3PFGDef']=Perc3D


#FreeThrowsOffensive
PercFTOf = []
PercFTOfo = soup.find_all(class_='9')
for eff in PercFTOfo:
    try:
        PercFTOf.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['FTOff']=PercFTOf


#FreeThrowsDeffensive
PercFTD = []
PercFTOD = soup.find_all(class_='10')
for eff in PercFTOD:
    try:
        PercFTD.append(eff.text[:len(eff.text)-len(eff.find(class_='lowrow').text)])
    except:
        pass
TeamData['FTDef']=PercFTD


Stats = pd.DataFrame(data = TeamData)
Stats.to_csv('FullSeasonStats_2010.csv', index = False)


In [362]:
Stats.to_csv('Games\FullSeasonStats_2010.csv', index = False)

In [379]:
#Remove Tournament seeding

years = list(range(2010,2022))
for year in years:
    filename = 'FullStats\FullSeasonStats_'+str(year)+'.csv'
    with open(filename,'r') as f:
        df = pd.read_csv(f)
    CorrectedName = []
    for nam in df['Name']:
        temp = ''
        Second = False
        NotFound=True
        for part in nam.split():
            if part.isdigit() != 1 and NotFound:
                if Second:
                    temp+=' '
                temp+=part
                Second=True
            else:
                NotFound=False
        CorrectedName.append(temp)
    df['Name'] = CorrectedName
    df.to_csv(filename, index = False)

In [386]:
#Retrieve stats from team's year
def GetTeamStats(TeamName,Year):
    Url ='https://www.barttorvik.com/team.php?team=' + str(TeamName).replace(' ','+') + '&year=' + str(Year) 
    TeamData = {}
    page = GetPageData(Url)
    soup = BeautifulSoup(page.content, 'html.parser')    
    return(soup)

#Scrape the opponent and +/-
def ScrapOpponent(Soup,year,team):
    Data = {}
    PlusMinus=[]
    Opponent=[]
    GameStats = Soup.find_all(class_='mobileout')
    i = 23
    while i < len(GameStats):
        try:
            PlusMinus.append(GameStats[i+18].text)
            Opponent.append(GameStats[i].text)
        except:
            pass
        i+=21
    Data['Opponent']=Opponent
    Data['PlusMinus']=PlusMinus
    Stats = pd.DataFrame(data = Data)
    filename = 'Games\\'+str(year)+'\\'+team+'_'+str(year)+'_Opponent.csv'
    Stats.to_csv(filename, index = False)
    return(0)

In [405]:
year = 2010
team = 'Grambling St.'
bean = df.Name
#print(bean)
tem = ['Gonzaga','Vdog',"Houston"]
for teams in tem:
    if 'Houston' in df.Name:
        print(teams)
#TeamData = GetTeamStats(team,year)
#ScrapOpponent(TeamData,year,team)

In [412]:
'Houston'.isin(df.Name)

AttributeError: 'str' object has no attribute 'isin'

In [383]:
years = list(range(2010,2022))
for year in years:
    #Get the relevant year's full stats
    filename = 'FullStats\FullSeasonStats_'+str(year)+'.csv'
    with open(filename,'r') as f:
        df = pd.read_csv(f)
    TeamData = GetTeamStats(team,year)
    ScrapOpponent(TeamData,year,team)

0

In [385]:
for i in range(0,100):
    print(df['Name'][i])

Gonzaga
Houston
Baylor
Michigan
Iowa
Illinois
Florida St.
Ohio St.
Alabama
San Diego St.
Wisconsin
West Virginia
Creighton
Villanova
Virginia
Colorado
Arkansas
Texas Tech
Purdue
Loyola Chicago
Kansas
Texas
USC
Tennessee
BYU
Florida
Connecticut
Oklahoma
Georgia Tech
Memphis
Maryland
LSU
Oklahoma St.
Utah St.
Duke
Missouri
Arizona
Rutgers
Seton Hall
Kentucky
Saint Louis
St. Bonaventure
Virginia Tech
Indiana
VCU
Clemson
North Carolina
Penn St.
UCLA
Mississippi
SMU
Xavier
Louisville
Mississippi St.
Oregon
Minnesota
Providence
Stanford
North Carolina St.
Michigan St.
Georgetown
Toledo
Richmond
Marquette
Syracuse
Wichita St.
North Texas
St. John's
Notre Dame
Boise St.
Buffalo
Auburn
Saint Mary's
Drake
Utah
Wright St.
UC Santa Barbara
Colorado St.
Northwestern
Davidson
Louisiana Tech
Nebraska
Pittsburgh
Dayton
Western Kentucky
Liberty
Arizona St.
Georgia
Washington St.
Colgate
Belmontvs.
Marshall
UC Riverside
Vanderbilt
Ohio
Furman
Pepperdine
Loyola Marymount
Oregon St.
UCF


In [203]:
dataMich = GetTeamStats('Michigan','2019')
relevant = dataMich.find_all(class_='teamOne')
dataStuff = relevant[0].find('script')
rawj = str(dataStuff)
j = rawj.split('var alldata = ')[1]
dataArray = j.rsplit('var gdata',1)[0]
tup = json.loads(dataArray[:34700])
for keys in tup:
    print(keys + ': ' + str(tup[keys]))

ftrd: [[18.6, 18.6, 18.6, 'Norfolk St.'], [5.8, 12.603603603604, 12.603603603604, 'Holy Cross'], [47.7, 22.566451612903, 22.566451612903, 'Villanova'], [9.4, 18.718721461187, 18.718721461187, 'George Washington'], [35.1, 22.101811594203, 22.101811594203, 'Providence'], [6.6, 19.443525179856, 19.295845697329, 'Chattanooga'], [19.7, 21.931164383562, 19.362034739454, 'North Carolina'], [9.7, 15.827419354839, 18.07376344086, 'Purdue'], [25.5, 18.873400673401, 18.807751937984, 'Northwestern'], [22.6, 16.57582781457, 19.214532871972, 'South Carolina'], [43.5, 23.01149825784, 21.004807692308, 'Western Michigan'], [26.7, 24.457142857143, 21.387892376682, 'Air Force'], [26, 28.362992125984, 21.708623087622, 'Binghamton'], [15.8, 26.170384615385, 21.274613402062, 'Penn St.'], [29.6, 27.78373015873, 21.816265060241, 'Indiana'], [11.5, 21.355805243446, 21.109988776655, 'Illinois'], [24.1, 21.021376811594, 21.280846560847, 'Northwestern'], [15.5, 19.017253521127, 20.946560319043, 'Wisconsin'], [26.

In [231]:
dataMich = GetTeamStats('Michigan','2019')
relevant = dataMich.find('tbody')
for rel in relevant:
    #print(rel.prettify())
    alldata = rel.find_all(class_='mobileout')
    for items in alldata:
        print(items.text)

NameError: name 'point_num' is not defined

In [273]:
dataMich = GetTeamStats('Michigan','2019')
Stadium = ['H','A','N']
Confrence = ['conf', 'notconf','post']
Rank = ['all','t50','t100']
Court=[]
Conf=[]
Rnk=[]
PlusMinus=[]
Opp=[]
for s in Stadium:
    for c in Confrence:
        for r in Rank:
            clas =c+' '+s+' '+r
            print(clas)
            classes = dataMich.find_all(class_=clas)
            for game in classes:
                Court.append(s)
                Conf.append(c)
                Rnk.append(r)
                stats = game.find_all(class_='mobileout')
                PlusMinus.append(stats[10].text)
                Opp.append(stats[1].text)


conf H all


IndexError: list index out of range

In [341]:
Court=[]
Conf=[]
Rnk=[]
PlusMinus=[]
Opp=[]
clas ='notconf A t50'
classes = dataMich.find_all(class_='mobileout')
#clas = dataMich.find(class_='teamFour')
las = dataMich.find_all(class_=clas)
i=23
dataVil = GetTeamStats('Villanova','2019')
Class2 = dataVil.find_all(class_='mobileout')
while i < len(Class2):
    try:
        print(Class2[i+18].text)
        print(Class2[i].text)
    except:
        print("EOF")
    i+=21


10.7
Morgan St.
20.5
Quinnipiac
-17.8
Michigan
0.8
Furman
11.7
Canisius
13.7
Oklahoma St.
1.6
Florida St.
-1.7
La Salle
0.3
Temple
8
Saint Joseph's
-4.7
Penn
-2.2
Kansas
4.1
Connecticut
-4.1
DePaul
13.6
Providence
-5.2
St. John's
0.7
Creighton
6.1
Xavier
4
Butler
9.3
Seton Hall
6.8
DePaul
2.7
Georgetown
-0.1
Creighton
-5
Marquette
3.4
Providence
8.1
St. John's
-7.6
Georgetown
0.4
Xavier
2.7
Marquette
4.9
Butler
-5.8
Seton Hall
3.5
Providence
-2.4
Xavier
2.4
Seton Hall
3.5
Saint Mary's
-17.9
Purdue
EOF
